# Price prediction using random forest machine learning

In [145]:
import yfinance as yf

start = start = '1950-01-01'
data = yf.download('^GSPC', start = start)
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1950-01-03,16.660000,16.660000,16.660000,16.660000,16.660000,1260000
1950-01-04,16.850000,16.850000,16.850000,16.850000,16.850000,1890000
1950-01-05,16.930000,16.930000,16.930000,16.930000,16.930000,2550000
1950-01-06,16.980000,16.980000,16.980000,16.980000,16.980000,2010000
1950-01-09,17.080000,17.080000,17.080000,17.080000,17.080000,2520000
...,...,...,...,...,...,...
2024-06-05,5314.479980,5354.160156,5297.640137,5354.029785,5354.029785,3591460000
2024-06-06,5357.799805,5362.350098,5335.359863,5352.959961,5352.959961,3609990000
2024-06-07,5343.810059,5375.080078,5331.330078,5346.990234,5346.990234,3692760000


We predict direction, not absolute price values as that is more important

In [146]:
data['Tomorrow'] = data['Close'].shift(-1)
data['Target'] = (data['Tomorrow'] > data['Close']).astype(int)
data = data.loc['1990-01-09':].copy()
data

,Open,High,Low,Close,Adj Close,Volume,Tomorrow,Target
Date,,,,,,,,
1990-01-09,353.829987,354.170013,349.609985,349.619995,349.619995,155210000,347.309998,0
1990-01-10,349.619995,349.619995,344.320007,347.309998,347.309998,175990000,348.529999,1
1990-01-11,347.309998,350.140015,347.309998,348.529999,348.529999,154390000,339.929993,0
1990-01-12,348.529999,348.529999,339.489990,339.929993,339.929993,183880000,337.000000,0
1990-01-15,339.929993,339.940002,336.570007,337.000000,337.000000,140590000,340.750000,1
...,...,...,...,...,...,...,...,...
2024-06-05,5314.479980,5354.160156,5297.640137,5354.029785,5354.029785,3591460000,5352.959961,0
2024-06-06,5357.799805,5362.350098,5335.359863,5352.959961,5352.959961,3609990000,5346.990234,0
2024-06-07,5343.810059,5375.080078,5331.330078,5346.990234,5346.990234,3692760000,5360.790039,1


In [65]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

model = RandomForestClassifier(random_state=1)

param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'min_samples_split': [2, 10, 50, 100]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='precision', n_jobs=-1)
grid_search.fit(train[predictors], train['Target'])

best_params = grid_search.best_params_

In [66]:
best_params

{'min_samples_split': 100, 'n_estimators': 500}

In [67]:
from sklearn.ensemble import RandomForestClassifier# as rfc

model = RandomForestClassifier(n_estimators = 300, min_samples_split = 100, random_state = 1)

train = data.iloc[:-100]
test = data.iloc[-100:]

predictors = ['Close', 'Volume', 'Open', 'High', 'Low']
model.fit(train[predictors], train['Target'])

RandomForestClassifier(min_samples_split=100, n_estimators=300, random_state=1)

In [68]:
from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [69]:
import pandas as pd

preds = pd.Series(preds, index = test.index)
preds

Date
2024-01-10    0
2024-01-11    0
2024-01-12    0
2024-01-16    1
2024-01-17    1
             ..
2024-05-28    0
2024-05-29    0
2024-05-30    0
2024-05-31    0
2024-06-03    0
Length: 100, dtype: int32

In [70]:
precision_score(test['Target'], preds)

0.5

In [71]:
test['Target']

Date
2024-01-10    0
2024-01-11    1
2024-01-12    0
2024-01-16    0
2024-01-17    1
             ..
2024-05-28    0
2024-05-29    0
2024-05-30    1
2024-05-31    1
2024-06-03    0
Name: Target, Length: 100, dtype: int32

In [75]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train['Target'])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index = test.index, name = 'Predictions')
    combined = pd.concat([test['Target'], preds], axis = 1)
    return combined

In [102]:
def backtest(data, model, predictors, start = 2500, step = 250):
    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i + step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)

In [77]:
predictions = backtest(data, model, predictors)

In [79]:
predictions['Predictions'].value_counts()

Predictions
0    3208
1    2958
Name: count, dtype: int64

In [80]:
precision_score(predictions['Target'], predictions['Predictions'])

0.5294117647058824

In [81]:
predictions['Target'].value_counts() / predictions.shape[0]

Target
1    0.535031
0    0.464969
Name: count, dtype: float64

In [147]:
horizons = [2, 5, 60, 250, 1000]
new_predictors = []

for horizon in horizons:
    rolling_averages = data.rolling(horizon).mean()
    
    ratio_column = f"Close_ratio_{horizon}"
    data[ratio_column] = data['Close'] / rolling_averages['Close']
    
    trend_column = f"Trend_{horizon}"
    data[trend_column] = data.shift(1).rolling(horizon).sum()['Target']
    #New_predictors.append(ratio_column)
    #new_predictors.append(trend_column)
    new_predictors += [ratio_column, trend_column]

In [148]:
data[new_predictors]

,Close_ratio_2,Trend_2,Close_ratio_5,Trend_5,Close_ratio_60,Trend_60,Close_ratio_250,Trend_250,Close_ratio_1000,Trend_1000
Date,,,,,,,,,,
1990-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-10,0.996685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-11,1.001753,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-12,0.987508,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-15,0.995672,0.0,0.978292,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-06-05,1.005889,2.0,1.012419,4.0,1.032468,33.0,1.134136,141.0,1.275185,531.0
2024-06-06,0.999900,1.0,1.007740,4.0,1.031673,32.0,1.132893,140.0,1.274254,530.0
2024-06-07,0.999442,0.0,1.003989,3.0,1.029921,32.0,1.130627,139.0,1.272156,530.0


In [149]:
data = data.dropna()

In [150]:
model = RandomForestClassifier(n_estimators = 300, min_samples_split = 50, random_state = 1)

In [151]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train['Target'])
    preds = model.predict_proba(test[predictors])[:, 1]
    preds[preds >= 0.6] = 1
    preds[preds < 0.6] = 0
    preds = pd.Series(preds, index = test.index, name = 'Predictions')
    combined = pd.concat([test['Target'], preds], axis = 1)
    return combined

In [152]:
predictions = backtest(data, model, new_predictors)

In [153]:
predictions['Predictions'].value_counts()

Predictions
0.0    4364
1.0     807
Name: count, dtype: int64

In [154]:
precision_score(predictions['Target'], predictions['Predictions'])

0.5700123915737298